In [ ]:
%load_ext autoreload
%autoreload 2

import seaborn as sns
import pandas as pd
import torch
import numpy as np
from interpretability import sample_from_hist, get_hist

In [ ]:
def shapley_value(x, b, i, func, hist=None):
    from itertools import chain, combinations
    from math import factorial
    def powerset(iterable):
        "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
        s = list(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

    n = len(x)
    l = list(range(n))
    del l[i]
    sv = 0

    if hist is None:
        b = torch.zeros(n)
    else:
        b = torch.tensor([sample_from_hist(hist[i]) for i in range(n)], dtype=torch.float32).flatten()

    for S in powerset(l):
        S = np.array(S).flatten()
        v1, v2 = b.clone(), b.clone()
        v1[i] = x[i]
        if len(S) != 0:
            v1[S] = x[S]
            v2[S] = x[S]
        const = factorial(len(S))*factorial(n-len(S)-1)/factorial(n)
        sv += const*(func(v1)-func(v2))
    return abs(sv)

In [ ]:
def func():
    n = 3
    t1 = np.concatenate([np.random.lognormal(mean=i, sigma=1, size=(n,1)) for i in range(4)], axis=1)
    t2 = np.random.lognormal(mean=2, sigma=1, size=(n,1))
    phi = np.concatenate((t1, t2), axis=1)
    phi = torch.tensor(phi)
    phi = torch.mean(phi, axis=0).flatten()
    phi = phi.float().flatten()
    def foo(x):
        return torch.sum(phi * x)
    return foo

def generate():
    t1 = np.random.normal(loc=0, scale=0.1, size=4)
    t2 = np.random.normal(loc=1, scale=0.1, size=1)
    x = np.concatenate((t1, t2))
    x = torch.tensor(x, dtype=torch.float32).flatten()
    return x

In [ ]:
hist_input = [get_hist(np.array([generate()[i].item() for _ in range(100)])) for i in range(5)]

In [ ]:
b = torch.zeros(5)
res = []
for _ in range(10):
    aux2 = []
    f = func()
    for _ in range(20):
        x = generate()
        aux = []
        for i in range(5):
            aux.append(shapley_value(x, b, i, f, None).item()) # Change this
        aux2.append(aux)
    res.append(aux2)
res = np.mean(np.array(res), axis=0)

In [ ]:
data = {"x": res.flatten(), "class": np.tile(range(5), 20)}
sns.boxplot(x="class", y="x", data=data)